In [1]:
import random
from numpy.random import choice
import re

In [2]:
rede = "redes/SC_Torque.sif"
motify = "motify/SC_Yeast_Complexes_SGD_filtrado.txt"
folder = "SC"

#rede = "redes/HomoSapiens_Torque.sif"
#motify = "motify/HS_Human_Complexes_CORUM_filtrado.txt"
#folder = "HS"

#rede = "redes/DM_Torque.sif"
#folder = "DM"
#motify = "motify/DM_Fly_Complexes_GO_filtrado.txt"

In [3]:
parent = dict()
rank = dict()

def make_set(vertice):
    parent[vertice] = vertice
    rank[vertice] = 0

def find(vertice):
    if parent[vertice] != vertice:
        parent[vertice] = find(parent[vertice])
    return parent[vertice]

def union(vertice1, vertice2):
    root1 = find(vertice1)
    root2 = find(vertice2)
    if root1 != root2:
        if rank[root1] > rank[root2]:
            parent[root2] = root1
	else:
	    parent[root1] = root2
	if rank[root1] == rank[root2]: rank[root2] += 1

def kruskal(graph):
    edges = list(graph['edges'])
    
    #random.shuffle(edges)
    edges.sort()
    #edges.reverse()
    # print edges
    
    for vertice in graph['vertices']:
        make_set(vertice)
        minimum_spanning_tree = set()

    for edge in edges:
        weight, vertice1, vertice2 = edge
        if find(vertice1) != find(vertice2):
            union(vertice1, vertice2)
            minimum_spanning_tree.add(edge)
            
    return  (minimum_spanning_tree)

In [4]:
file_ = open(rede, "r")
rows = file_.readlines()

edges = []

#store value in format vertice_name => color
vertices_colors = dict()
vertice_color = 0

for row in rows:
    data = row.split()
    
    start_vertice = data[0]
    weight = float(data[1])
    end_vertice = data[2]
    
    edges.append((weight, start_vertice, end_vertice))
    
    vertices_colors[start_vertice] = vertice_color
    vertice_color += 1 
    
    vertices_colors[end_vertice] = vertice_color
    vertice_color += 1
    
vertices = list(vertices_colors.keys())
print len(vertices)


5414


In [5]:
vertices_id = dict()

counter = 0

for vertice_ in vertices:
    vertices_id[vertice_] = counter
    counter = counter + 1
print counter

5414


In [6]:
#Trocar as cores
for edge in edges:
    weight = edge[0]
    start = edge[1]
    end = edge[2]
    
    #probabilidade de pegar a cor do primeiro
    should_change_color = choice(a = [True, False], p = [weight, 1 - weight])
    
    if should_change_color:
        
        should_get_first_color = choice(a = [True, False], p = [0.5, 0.5])
        
        if should_get_first_color:
            vertices_colors[end] = vertices_colors[start]
        else: 
            vertices_colors[start] = vertices_colors[end]

In [7]:
#rerotular
color = dict()
color_count = 0

for vertice, vertice_color in vertices_colors.iteritems():
    if vertice_color not in color:
        color[vertice_color] = color_count
        color_count = color_count + 1    
           
            
    vertices_colors[vertice] = color[vertice_color]
    
color_count = color_count + 1
print color_count

2491


In [8]:
#cria arquivos de vertices

vertices_colors_file = file(folder + "/vertices_colors.csv", "w+")

vertices_color_content = []

for vertice, vertice_color in vertices_colors.iteritems():
    vertices_color_content.append(str(vertice_color + 1))
    
print "numero de vertices: ", len(vertices_color_content)
    
vertices_colors_file.write(",".join(vertices_color_content))

vertices_colors_file.close()

file_after = file(folder + "/vertices_colors.csv", "r")
lines = file_after.readlines()

print "depois de creia o arquivo: ", len(lines[0].split(","))
file_after.close()


numero de vertices:  5414
depois de creia o arquivo:  5414


In [9]:
#cria lista de adjacencia
graph = {
    'vertices': vertices,
    'edges': set(edges)
}

edges_selected = kruskal(graph)

number_of_vertices = len(graph["vertices"])

adjacency_list = [[] for i in xrange(number_of_vertices)]

for edge in edges_selected:
    start_vertice = edge[1]
    end_vertice = edge[2]
    
    adjacency_list[vertices_id[start_vertice]].append(str(vertices_id[end_vertice] + 1))
    adjacency_list[vertices_id[end_vertice]].append(str(vertices_id[start_vertice] + 1))

In [10]:
#criar arquivo de lista de adjacencia
edges_file = file(folder + "/edges.csv", "w+")

adjacency_list_text = [[] for i in xrange(number_of_vertices)]

for i in xrange(number_of_vertices):
    adjacency_list_text[i] = " ".join(str(v) for v in adjacency_list[i])
    
print len(adjacency_list_text)
    
edges_file.write("\n".join(adjacency_list_text))
edges_file.close()

files_after_ = file(folder + "/edges.csv", "r")
lines_ = files_after_.readlines()

print "number after: ", len(lines_[0].split("\n"))
files_after_.close()


5414
number after:  2


In [11]:
#cria arquivo de motify
motify_file = open(motify, "r")
motify_lines = motify_file.readlines()

for i in xrange(len(motify_lines)):    
    counter = 0
    
    motify_sample = motify_lines[i]
    
    motify_parse = re.split(r'\t+', motify_sample)
    motify_description = motify_parse[1].split()
    
    i_str = str(i + 1)

    color_frequency = ["0"] * color_count
    total_color = 0

    for vertice_ in motify_description:
        
        if vertice_ in vertices_colors: 
            color = vertices_colors[vertice_]
            
            frequency = int(color_frequency[color])
            
            if frequency == 0:
                total_color = total_color + 1
            
            color_frequency[color] = str(frequency + 1)
        else:
            counter = counter + 1
    
    total_vertices = len(motify_description) - counter
    motify_frequency_file = open(folder + "/motify-" + i_str + "-" + str(total_vertices) + "-" + str(total_color) + ".csv", "w+")

    motify_frequency_file.write(",".join(color_frequency))
    motify_frequency_file.close()
    
    file_after__ = open(folder + "/motify-" + i_str + "-" + str(total_vertices) + "-" + str(total_color) + ".csv", "r")
    files_lines__ = file_after__.readlines()
    print "value after: ", len(files_lines__[0].split(","))
    file_after__.close()

value after:  2491
value after:  2491
value after:  2491
value after:  2491
value after:  2491
value after:  2491
